In [ ]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
import random
from xgboost import XGBClassifier

In [ ]:

train = sp500.iloc[:-100]
test = sp500.iloc[-100:]


etas = [0.01,0.05,0.10,0.20,0.30,0.40]

params = {
    'max_depth': [5, 10, 15, 20,25],
    'subsample': [0.5,0.6,0.7,0.8,0.9,1.0],
    'colsample_bytree': [0.5,0.6,0.7,0.8,0.9,1.0],
    'gamma' : [0.00,0.05,0.10,0.15,0.20],
    'scale_pos_weight' : [30,40,50,300,400,500,600,700], 
}

iSol = {}
for key,values in params.items():
    iSol[key] = random.choice(values)

cSol = iSol
tStart = 100
tEnd = 1e-3
nLoops = 30 


frac = (tEnd/tStart)**(1.0/(nLoops-1.0))



def predict(train, test, predictors, model):
    model.fit(train[predictors], train["Target"])
    preds = model.predict(test[predictors])
    preds = pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test["Target"], preds], axis=1)
    return combined


def backtest(data, model, predictors, start=2500, step=250):
    all_predictions = []

    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+step)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)
    
    return pd.concat(all_predictions)





def objective_function(cSol):
    
    model = XGBClassifier(**cSol)
    
 
    predictors = ["Close", "Volume", "Open", "High", "Low"]
    model.fit(train[predictors], train["Target"])
    
    
    predictions = backtest(sp500, model, predictors)
    
    return precision_score(predictions["Target"], predictions["Predictions"])


def simulated_annealing(T, frac, n):
   
    current_solution = {}

    

    for key,values in params.items():
        current_solution[key] = random.choice(values)
        current_solution.update({'eta':0.001})

    
    current_obj_val = objective_function(current_solution)
    
    
    for i in range(n):
        
        new_solution = {}
        for  eta in etas:
            for key,values in params.items():
                new_solution[key] = random.choice(values)
                new_solution.update({'eta':eta})
            
            
            new_obj_val = objective_function(new_solution)

            if new_obj_val>current_obj_val:
                current_obj_val = new_obj_val
                current_solution = new_solution
            elif new_obj_val <= current_obj_val:
                
                p = np.exp(-(current_obj_val - new_obj_val) / T)
                
                if np.random.uniform(0, 1) < p:
                    current_solution = new_solution
                    current_obj_val = new_obj_val
            
            
            T *= frac
            
            
            if T < tEnd:
                break
            print("Current Temp", T)
            print("Current Hyperparameters: ",current_solution)
            print("Current Objective Value: ",current_obj_val)
            
    
    
    return current_solution, current_obj_val,T


best_solution, best_obj_val = simulated_annealing(tStart, frac, nLoops)


print("Best Hyperparameters : ", best_solution)
print("Best Objective Function Value : ",best_obj_val)


